In [2]:
import numpy as np
import itertools
import sys
sys.path.insert(0, '../CClassCSemigroups/')
import CsemigroupsCPP as CPP

In [3]:
def PrecGradLex(a,b): # Check if A<B\n",
    sumA = np.array(a).sum()
    sumB = np.array(b).sum()
    if sumA < sumB:
        return True
    elif sumA > sumB:
        return False
    else:
        c = np.array(b)-np.array(a)
        for x in c:
            if x > 0:
                return True
            if x < 0:
             return False

In [4]:
def GetSemigroupFromGaps(gaps,eq,order): # Note: Last gap is the frobenius element.
    
    frob = list(MaximumOrder(gaps,order)) # El list es porque si no, se contamina 'gaps' por algún motivo

    mfrob = max(frob)
    for i in range(len(frob)):
        if frob[i] == 0:
            frob[i] = mfrob # Arreglo temporal!!!
    
    bound =  [3*frob[i] for i in range(len(frob))]### <--- COMPROBAR SI ESTA COTA SE PUEDE MEJORAR!!!
    cone = []
    it = itertools.product(*[range(i+1) for i in bound])
    for x in it:
        if CPP.belongCone(list(x),eq) and not CPP.allZero(list(x)):
            cone.append(list(x))
    print("Cone = ",cone)
    print("gaps =",gaps)
    for x in gaps:
        cone.remove(x)
    cs = CPP.Csemigroup(cone)
    return cs

In [5]:
def MaximumOrder(v,order):
    maximum = v[0]
    for x in v:
        if order(maximum,x):
            maximum = x
    return maximum

In [6]:
def MinimumOrder(v,order):
    minimum = v[0]
    for x in v:
        if order(x,minimum):
            minimum = x
    return minimum

In [7]:
def ComputeCone(bound,eq):
    cone = []
    it = itertools.product(*[range(i+1) for i in bound])
    for x in it:
        if CPP.belongCone(list(x),eq):
            cone.append(list(x))
    return cone

In [8]:
def DeleteDuplicates(v):
    w = []
    for x in v:
        if x not in w:
            w.append(x)
    return w

#DeleteDuplicates([1, 3, 4, 2, 1, 6, 7, 3])

In [9]:
def Subset(v,w):
    for x in v:
        if x not in w:
            return False
    return True

In [10]:
def ComputeMaximimumPerCoordinate(v):
    n = len(v[0])
    maximum  = []
    for i in range(n):
        aux = 0
        for x in v:
            if x[i] > aux:
                aux = x[i]
        maximum.append(aux)
    return maximum

In [11]:
def ComputeD(eqCone, X):
    bound = ComputeMaximimumPerCoordinate(X)
    points = ComputeCone(bound, eqCone)
    points = [x for x in points if not CPP.allZero(x)]
    candidates = list(points)
    D = []
    maxN = max(bound)
    for i in range(1,maxN+1):
        candidates = [[i*x[j] for j in range(len(x))] for x in points]
        #print("i = ",i,"candidates = ",candidates)
        for j in range(len(candidates)):
            if candidates[j] in X:
                D.append(points[j])
    #print(points)
    #print(X)
    return D

#ComputeD([[0, 1], [2, -1]], [[2,0],[2,1]])

In [12]:
ComputeD([[0,1],[1,-1]],[[1, 1], [3, 0], [3, 1], [3, 2], [5, 1], [5, 2]])

[[1, 1], [3, 0], [3, 1], [3, 2], [5, 1], [5, 2], [1, 0]]

In [13]:
def ConeMinusSet(cone, X, order):
    genCone = cone.GetGenerators()
    eqCone = cone.GetHyperplanes()
    if Subset(X,genCone):
        return GetSemigroupFromGaps(X,eqCone,order)
    D = DeleteDuplicates(ComputeD(eqCone,X))
    print("D=",D)
    x1 = MinimumOrder(X,order)
    if not Subset(X,D) or not Subset(D,X):
        raise Exception("C\X is not a C-semigroup") 
    gen = [x for x in genCone]
    t = len(X)
    gaps = []
    aux = X
    for i in range(t):
        print("\n t=",i)
        print("Estudiamos ",X[i])
        if X[i] not in gen:
            raise Exception("C\X is not a C-semigroup")
        gaps.append(X[i])
        print("gaps = ",gaps)
        gen = GetSemigroupFromGaps(gaps,eqCone,order).GetGenerators()
        print("Gen =",gen)
        aux = [x for x in aux if x != X[i]]
        if Subset(aux,gen):
            return GetSemigroupFromGaps(gaps+aux,eqCone,order)

In [14]:
#eq = [[0, 1], [2, -1]]
gen = [[1,0],[1,1],[1,2]]
X1 = [[1,0], [1,1]]
X2 = [[2,0],[2,1]]
X3 = [[1,0],[1,1],[1,2],[2,0],[2,1],[2,2],[2,3],[2,4]]
cone = CPP.Csemigroup(gen)
ConeMinusSet(cone, X3, PrecGradLex).GetGenerators()

D= [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4]]

 t= 0
Estudiamos  [1, 0]
gaps =  [[1, 0]]
Cone =  [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [2, 3], [3, 0], [3, 1], [3, 2], [3, 3]]
gaps = [[1, 0]]
Gen = [[2, 0], [3, 0], [1, 1], [2, 1], [1, 2]]

 t= 1
Estudiamos  [1, 1]
gaps =  [[1, 0], [1, 1]]
Cone =  [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [2, 3], [3, 0], [3, 1], [3, 2], [3, 3]]
gaps = [[1, 0], [1, 1]]
Gen = [[2, 0], [3, 0], [2, 1], [3, 1], [1, 2], [2, 2], [2, 3]]

 t= 2
Estudiamos  [1, 2]
gaps =  [[1, 0], [1, 1], [1, 2]]
Cone =  [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6]]
gaps = [[1, 0], [1, 1], [1, 2]]
Gen = [[2, 0], [3, 0], [2, 1], [3, 1], [2, 2], [3, 2], [2, 3], [3, 3], [2, 4], [3, 4], [3, 5], [3, 6]]
Cone =  [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4

[[3, 0],
 [4, 0],
 [5, 0],
 [3, 1],
 [4, 1],
 [5, 1],
 [3, 2],
 [4, 2],
 [5, 2],
 [3, 3],
 [4, 3],
 [5, 3],
 [3, 4],
 [4, 4],
 [5, 4],
 [3, 5],
 [4, 5],
 [5, 5],
 [3, 6],
 [4, 6],
 [5, 6],
 [4, 7],
 [5, 7],
 [4, 8],
 [5, 8],
 [5, 9],
 [5, 10]]